# Dialogue Evaluation
###### Created by Weber Huang 

In [1]:
import os
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import numpy as np
from collections import Counter
from sklearn import preprocessing
import jieba
import re

In [2]:
train = pd.read_csv('C:\\Users\\doudi\\OneDrive\\Documents\\ntcir15\\Dataset\\New_DialEval-1\\train_baidu_cn.csv', encoding='utf_8')
dev = pd.read_csv('C:\\Users\\doudi\\OneDrive\\Documents\\ntcir15\\Dataset\\New_DialEval-1\\dev_baidu_cn.csv', encoding='utf_8')

In [3]:
test = pd.read_csv('C:\\Users\\doudi\\OneDrive\\Documents\\ntcir15\\Dataset\\New_DialEval-1\\test_w2v_cn.csv', encoding='utf_8')

In [4]:
train['Baidu_str'] = train['Baidu_str'].fillna('[]')
dev['Baidu_str'] = dev['Baidu_str'].fillna('[]')
test['W2V_str'] = test['W2V_str'].fillna('[]')

In [5]:
def str_to_list(dataframe, string): 
    l = []
    n = []
    for i in dataframe[string]:
        if i != '[]':
            
            l.append([float(j) for j in str(i).split()])
        else:
            l.append(n)
    return l

In [6]:
train["W2V"] = str_to_list(train, 'W2V_str')
dev["W2V"] = str_to_list(dev, 'W2V_str')
test["W2V"] = str_to_list(test, 'W2V_str')

In [7]:
test.loc[50]

id            4241750585847250
sender                customer
sender_num                   0
texts                        *
round                        3
W2V                         []
W2V_str                     []
Name: 50, dtype: object

In [8]:
from scipy import stats
import tensorflow as tf

def normalize(pred, truth):
    """ convert inputs to np.array and make sure
    inputs are normalized probability distributions
    """
    if len(pred) != len(truth):
        raise ValueError("pred and truth have different lengths")
    if len(pred) == 0 or len(truth) == 0:
        raise ValueError("pred or truth are empty")

    pred, truth = np.asarray(pred), np.asarray(truth)
    if not ((pred >= 0).all() and (truth >= 0).all()):
        raise ValueError("probability distribution should not be negative")
    pred, truth = pred / pred.sum(), truth / truth.sum()
    return pred, truth

def jensen_shannon_div(pred, truth, base=2):
    ''' JSD: Jensen-Shannon Divergence
    '''
    pred, truth = normalize(pred, truth)
    m = 1. / 2 * (pred + truth)
    return (stats.entropy(pred, m, base=base)
            + stats.entropy(truth, m, base=base)) / 2.

def root_normalized_squared_error(pred, truth):
    """ RNSS: Root Normalised Sum of Squares
    """

    def squared_error(pred, truth):
        return ((pred - truth) ** 2).sum()

    pred, truth = normalize(pred, truth)
    return np.sqrt(squared_error(pred, truth) / 2)

def jsd_custom_loss(y_true, y_pred):
            
    # calculate loss, using y_pred
    ''' JSD: Jensen-Shannon Divergence
    '''
#     y_pred, y_true = normalize(y_pred, y_true)
    m = 1. / 2 * (y_pred + y_true)
    # loss = (stats.entropy(y_pred, m, base=2) + stats.entropy(y_true, m, base=2)) / 2.
    # tf.keras.losses.KLD()
    loss = (tf.keras.losses.KLD(y_pred, m) + tf.keras.losses.KLD(y_true, m)) / 2.
    return loss
  

C:\Users\doudi\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\doudi\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\doudi\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\doudi\AppData\Roaming\Python\Python37\site-

In [10]:
# split from sender
train_c = train[train.sender=='customer']
train_h = train[train.sender=='helpdesk']
dev_c = dev[dev.sender=='customer']
dev_h = dev[dev.sender=='helpdesk']
test_c = test[test.sender=='customer']
test_h = test[test.sender=='helpdesk']

In [11]:
from sklearn.model_selection import train_test_split 
from sklearn import metrics
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils
from keras import optimizers
from keras.layers.merge import concatenate
from keras.models import Sequential, Model
from keras.layers import Input
from keras.layers import Dense, Embedding
from keras.layers import Convolution1D, Flatten, Dropout, MaxPool1D, GlobalAveragePooling1D
from keras.layers import concatenate, Bidirectional
from keras import initializers
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.embeddings import Embedding
# from keras.layers.recurrent import SimpleRNN
from keras.layers.recurrent import LSTM
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [12]:
# === customer
c_X_train = train_c.filter(['round','texts'])
c_X_test = dev_c.filter(['round','texts'])
    

y_train_c = train_c.filter(['CNUG0','CNUG','CNUG*','CNaN'])
y_test_c = dev_c.filter(['CNUG0','CNUG','CNUG*','CNaN'])

# y_train_h = train.filter(['HNUG','HNUG*','HNaN'])
# y_test_h = dev.filter(['HNUG','HNUG*','HNaN'])

c_X1_train = c_X_train['texts']
# c_X1_train = [str (item) for item in c_X1_train]
c_X1_test = c_X_test['texts']
# c_X1_test = [str (item) for item in c_X1_test]

c_X2_train = c_X_train[['round']].values
c_X2_test = c_X_test[['round']].values

c_token = Tokenizer(num_words = 20000)
c_token.fit_on_texts(c_X1_train)
c_vocab = c_token.word_index
print(c_token.document_count)

c_x_train_seq = c_token.texts_to_sequences(c_X1_train)
c_x_test_seq = c_token.texts_to_sequences(c_X1_test)
c_X1_train = sequence.pad_sequences(c_x_train_seq, maxlen = 350)
c_X1_test = sequence.pad_sequences(c_x_test_seq, maxlen = 350)

8500


In [13]:
# === helpdesk
h_X_train = train_h.filter(['round','texts'])
h_X_test = dev_h.filter(['round','texts'])
    

# y_train_c = train_c.filter(['CNUG','CNUG*','CNUG0','CNaN'])
# y_test_c = dev_c.filter(['CNUG','CNUG*','CNUG0','CNaN'])

y_train_h = train_h.filter(['HNUG','HNUG*','HNaN'])
y_test_h = dev_h.filter(['HNUG','HNUG*','HNaN'])

h_X1_train = h_X_train['texts']
# h_X1_train = [str (item) for item in h_X1_train]
h_X1_test = h_X_test['texts']
# h_X1_test = [str (item) for item in h_X1_test]

h_X2_train = h_X_train[['round']].values
h_X2_test = h_X_test[['round']].values

h_token = Tokenizer(num_words = 20000)
h_token.fit_on_texts(h_X1_train)
h_vocab = h_token.word_index
print(h_token.document_count)

h_x_train_seq = h_token.texts_to_sequences(h_X1_train)
h_x_test_seq = h_token.texts_to_sequences(h_X1_test)
h_X1_train = sequence.pad_sequences(h_x_train_seq, maxlen = 350)
h_X1_test = sequence.pad_sequences(h_x_test_seq, maxlen = 350)

6900


> Word2Vec vector

In [14]:
def to_array(dataframe, length, string, dim=256):   
    train_vec = np.zeros(shape=(length,dim))
#     print(train_vec.shape)
    for i in range(len(dataframe[string])):
        if dataframe[string].iloc[i] != []:
#             print(type(dataframe[string].iloc[i]))
    #         print(i)
            train_vec[i] = dataframe[string].iloc[i]
        else:
            continue
    print(train_vec.shape)
    
    return train_vec

train_vec_c = to_array(train_c, len(train_c), 'W2V', dim=300)
train_vec_h = to_array(train_h, len(train_h), 'W2V', dim=300)
dev_vec_c = to_array(dev_c, len(dev_c), 'W2V', dim=300)
dev_vec_h = to_array(dev_h, len(dev_h), 'W2V', dim=300)
test_vec_c = to_array(test_c, len(test_c), 'W2V', dim=300)
test_vec_h = to_array(test_h, len(test_h), 'W2V', dim=300)

(8500, 300)
(6900, 300)
(975, 300)
(780, 300)
(714, 300)
(576, 300)


In [15]:
train_vec_c = np.reshape(train_vec_c, (train_vec_c.shape[0],1, train_vec_c.shape[1]))
train_vec_h = np.reshape(train_vec_h, (train_vec_h.shape[0],1, train_vec_h.shape[1]))
dev_vec_c = np.reshape(dev_vec_c, (dev_vec_c.shape[0],1,dev_vec_c.shape[1]))
dev_vec_h = np.reshape(dev_vec_h, (dev_vec_h.shape[0],1,dev_vec_h.shape[1]))
test_vec_c = np.reshape(test_vec_c, (test_vec_c.shape[0],1,test_vec_c.shape[1]))
test_vec_h = np.reshape(test_vec_h, (test_vec_h.shape[0],1,test_vec_h.shape[1]))

In [16]:
# === customer
def lstm_C(X1_train, X2_train, X1_test, X2_test, y_train, y_test, loss='categorical_crossentropy'):
     
#     main_input = Input(shape=(350,), dtype='float64')
    main_input = Input(shape=(1,300))
    sub_input = Input(shape=(1,))
    
#     embed = Embedding(output_dim=300,input_dim=20000,input_length=350)(main_input)
#     dropout_1 = Dropout(0.35)(embed)
    lst = Bidirectional(LSTM(units=256,return_sequences=True))(main_input)
    lst2 = Bidirectional(LSTM(units=128))(lst)
    merge = concatenate([lst2, sub_input])
    dense_1 = Dense(units=64,activation='relu')(merge)
    dropout_1 = Dropout(0.35)(dense_1)
#     dense_2 = Dense(units=32,activation='relu')(dropout_1)
    dense_3 = Dense(units=16,activation='relu')(dropout_1)
    output = Dense(units=4,activation='softmax')(dense_3)


    model = Model(inputs=[main_input, sub_input], outputs=output)
    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
    print(model.summary())

    # checkpoint
    filepath="C:\\Users\\doudi\\OneDrive\\Documents\\ntcir15\\Model file\\weights(c).best.hdf5"
    checkpoint= ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, 
                                save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    
    train_history = model.fit(x=[X1_train, X2_train], y=y_train, epochs=50, 
                              batch_size=128, callbacks=callbacks_list, verbose=1, validation_split=0.2)

    score = model.evaluate(x=[X1_test, X2_test], y=y_test, verbose=1)

    print("Test Score:", score[0])
    print("Test Accuracy:", score[1])

    pre_probability = model.predict(x=[X1_test, X2_test])
    predicted = pre_probability.argmax(axis=-1)
    
    return model, train_history, pre_probability

In [17]:
# === helpdesk
def lstm_H(X1_train, X2_train, X1_test, X2_test, y_train, y_test, loss='categorical_crossentropy'):
     
#     main_input = Input(shape=(350,), dtype='float64')
    main_input = Input(shape=(1,300))
    sub_input = Input(shape=(1,))
    
#     embed = Embedding(output_dim=300,input_dim=20000,input_length=350)(main_input)
#     dropout_1 = Dropout(0.35)(embed)
    lst = Bidirectional(LSTM(units=256,return_sequences=True))(main_input)
    lst2 = Bidirectional(LSTM(units=128))(lst)
    merge = concatenate([lst2, sub_input])
    dense_1 = Dense(units=64,activation='relu')(merge)
    dropout_1 = Dropout(0.35)(dense_1)
#     dense_2 = Dense(units=32,activation='relu')(dropout_1)
    dense_3 = Dense(units=16,activation='relu')(dropout_1)
    output = Dense(units=3,activation='softmax')(dense_3)




    model = Model(inputs=[main_input, sub_input], outputs=output)
    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
    print(model.summary())

    # checkpoint
    filepath="C:\\Users\\doudi\\OneDrive\\Documents\\ntcir15\\Model file\\weights(h).best.hdf5"
    checkpoint= ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, 
                                save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    
    train_history = model.fit(x=[X1_train, X2_train], y=y_train, epochs=50, 
                              batch_size=128, callbacks=callbacks_list, verbose=1, validation_split=0.2)

    score = model.evaluate(x=[X1_test, X2_test], y=y_test, verbose=1)

    print("Test Score:", score[0])
    print("Test Accuracy:", score[1])

    pre_probability = model.predict(x=[X1_test, X2_test])
    predicted = pre_probability.argmax(axis=-1)
    
    return model, train_history, pre_probability

In [27]:
lstm_c_model, lstm_c_history, lstm_c_pred = lstm_C(train_vec_c, c_X2_train, dev_vec_c, 
                                               c_X2_test, y_train_c, y_test_c, 
                                               loss = jsd_custom_loss)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1, 300)       0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 1, 512)       1140736     input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 256)          656384      bidirectional_1[0][0]            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1)            0                                            
____________________________________________________________________________________________

6800/6800 [==============================] - 6s 826us/step - loss: 0.0264 - accuracy: 0.8872 - val_loss: 0.0286 - val_accuracy: 0.8876

Epoch 00023: val_accuracy did not improve from 0.89765
Epoch 24/50
6800/6800 [==============================] - 6s 826us/step - loss: 0.0259 - accuracy: 0.8856 - val_loss: 0.0296 - val_accuracy: 0.8929

Epoch 00024: val_accuracy did not improve from 0.89765
Epoch 25/50
6800/6800 [==============================] - 6s 825us/step - loss: 0.0261 - accuracy: 0.8909 - val_loss: 0.0290 - val_accuracy: 0.8841

Epoch 00025: val_accuracy did not improve from 0.89765
Epoch 26/50
6800/6800 [==============================] - 6s 824us/step - loss: 0.0254 - accuracy: 0.8903 - val_loss: 0.0294 - val_accuracy: 0.8853

Epoch 00026: val_accuracy did not improve from 0.89765
Epoch 27/50
6800/6800 [==============================] - 6s 816us/step - loss: 0.0254 - accuracy: 0.8900 - val_loss: 0.0288 - val_accuracy: 0.8829

Epoch 00027: val_accuracy did not improve from 0.897

In [28]:
lstm_h_model, lstm_h_history, lstm_h_pred = lstm_H(train_vec_h, h_X2_train, dev_vec_h, 
                                               h_X2_test, y_train_h, y_test_h,  
                                               loss = jsd_custom_loss)

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 1, 300)       0                                            
__________________________________________________________________________________________________
bidirectional_3 (Bidirectional) (None, 1, 512)       1140736     input_3[0][0]                    
__________________________________________________________________________________________________
bidirectional_4 (Bidirectional) (None, 256)          656384      bidirectional_3[0][0]            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 1)            0                                            
____________________________________________________________________________________________

5520/5520 [==============================] - 3s 577us/step - loss: 0.0199 - accuracy: 0.8835 - val_loss: 0.0229 - val_accuracy: 0.8826

Epoch 00025: val_accuracy did not improve from 0.88551
Epoch 26/50
5520/5520 [==============================] - 3s 570us/step - loss: 0.0199 - accuracy: 0.8857 - val_loss: 0.0230 - val_accuracy: 0.8768

Epoch 00026: val_accuracy did not improve from 0.88551
Epoch 27/50
5520/5520 [==============================] - 3s 576us/step - loss: 0.0197 - accuracy: 0.8833 - val_loss: 0.0228 - val_accuracy: 0.8790

Epoch 00027: val_accuracy did not improve from 0.88551
Epoch 28/50
5520/5520 [==============================] - 3s 571us/step - loss: 0.0197 - accuracy: 0.8853 - val_loss: 0.0224 - val_accuracy: 0.8804

Epoch 00028: val_accuracy did not improve from 0.88551
Epoch 29/50
5520/5520 [==============================] - 3s 557us/step - loss: 0.0195 - accuracy: 0.8844 - val_loss: 0.0233 - val_accuracy: 0.8717

Epoch 00029: val_accuracy did not improve from 0.885

#### 6.2 Output by rbind

In [29]:
# predict single row
def padding_single_c(dev):
    X_test = dev.filter(['round','texts'])
    X1_test = X_test['texts']
#     X1_test = [str (item) for item in X1_test]
    X2_test = X_test[['round']].values
    
    x_test_seq = c_token.texts_to_sequences([X1_test])
    X1_test = sequence.pad_sequences(x_test_seq, maxlen = 350)
    
    return X1_test, X2_test

def padding_single_h(dev):
    X_test = dev.filter(['round','texts'])
    X1_test = X_test['texts']
#     X1_test = [str (item) for item in X1_test]
    X2_test = X_test[['round']].values
    
    x_test_seq = h_token.texts_to_sequences([X1_test])
    X1_test = sequence.pad_sequences(x_test_seq, maxlen = 350)
    
    return X1_test, X2_test

> word2Vec

In [18]:
def to_array_2(dataframe, length=1, dim=300):   
    train_vec = np.zeros(shape=(length,dim))
    if dataframe['W2V'] != []:
        train_vec = np.array(dataframe['W2V'])
    
#     print(train_vec.shape)
    train_vec = train_vec.reshape(1,300)
    
    return np.array(train_vec)

def padding_single_w2v(dev):
    X_test = dev.filter(['round','W2V'])
    X1_test = to_array_2(X_test)
    X1_test = np.reshape(X1_test, (X1_test.shape[0],1,X1_test.shape[1]))
    X2_test = X_test[['round']].values
    return X1_test, X2_test

In [33]:
from itertools import chain

# input the development dataframe and the method
# for current models (loss = jsd):
# model_1 = customer model (CNN_c_model, lstm_c_model)
# model_2 = helpdesk model (CNN_h_model, lstm_h_model)
def Generate_submission(dev, model_1, model_2):
    Id_list = dev['id'].unique()
    C_nugget = ['CNUG','CNUG*','CNUG0','CNaN']
    H_nugget = ['HNUG','HNUG*','HNaN']

    final = []
    
    # go through each Id first
    for Id in tqdm(Id_list):  
        result = []
        
        for i in range(len(dev)):
            
            # if Id is match than predict the prob_distribution and zip it as dictionary 
            if dev['id'][i] == Id:
                if dev.iloc[i, 1] == 'customer':
                    t1, t2 = padding_single_c(dev.iloc[i])
                    t2 = np.array(t2).reshape(1,1)
                    cus_prob = model_1.predict(x=[t1, t2])
                    cus_prob = cus_prob.tolist()
                    cus_prob = list(chain(*cus_prob))
                    dict_c = dict(zip(C_nugget, cus_prob))
                    result.append(dict_c)
                else:
                    t3, t4 = padding_single_h(dev.iloc[i])
                    t4 = np.array(t4).reshape(1,1)
                    help_prob = model_2.predict(x=[t3, t4])
                    help_prob = help_prob.tolist()
                    help_prob = list(chain(*help_prob))
                    dict_h = dict(zip(H_nugget, help_prob))
                    result.append(dict_h)
            # if Id isn't match than continue until it match or switch to new Id
            else:
                continue
        
        # Submission form
        dict1 = {'nugget':result,'id':Id}
        final.append(dict1)
        
    return final

> Word2Vec

In [19]:
from itertools import chain
def Generate_submission_w2v(dev, model_1, model_2):
    dev['id'] = dev['id'].apply(str)
    Id_list = dev['id'].unique()
    C_nugget = ['CNUG0','CNUG','CNUG*','CNaN']
    H_nugget = ['HNUG','HNUG*','HNaN']

    final = []
    
    # go through each Id first
    for Id in tqdm(Id_list):  
        result = []
        
        for i in range(len(dev)):
            
            # if Id is match than predict the prob_distribution and zip it as dictionary 
            if dev['id'][i] == Id:
                if dev.iloc[i, 1] == 'customer':
                    print(i)
                    t1, t2 = padding_single_w2v(dev.iloc[i])
                    t2 = np.array(t2).reshape(1,1)
                    cus_prob = model_1.predict(x=[t1, t2])
                    cus_prob = cus_prob.tolist()
                    cus_prob = list(chain(*cus_prob))
                    dict_c = dict(zip(C_nugget, cus_prob))
                    result.append(dict_c)
                else:
                    print(i)
                    t3, t4 = padding_single_w2v(dev.iloc[i])
                    t4 = np.array(t4).reshape(1,1)
                    help_prob = model_2.predict(x=[t3, t4])
                    help_prob = help_prob.tolist()
                    help_prob = list(chain(*help_prob))
                    dict_h = dict(zip(H_nugget, help_prob))
                    result.append(dict_h)
            # if Id isn't match than continue until it match or switch to new Id
            else:
                continue
        
        # Submission form
        dict1 = {'nugget':result,'id':Id}
        final.append(dict1)
        
    return final

In [20]:
# final = Generate_submission(dev, lstm_c_model, lstm_h_model)
from keras.models import load_model
path_1="C:\\Users\\doudi\\OneDrive\\Documents\\ntcir15\\Model file\\weights(c).best.hdf5"
path_2="C:\\Users\\doudi\\OneDrive\\Documents\\ntcir15\\Model file\\weights(h).best.hdf5"
model_c = load_model(path_1,custom_objects={'jsd_custom_loss': jsd_custom_loss})
model_h = load_model(path_2,custom_objects={'jsd_custom_loss': jsd_custom_loss})

model_c.compile(optimizer='adam', loss=jsd_custom_loss, metrics=['accuracy'])
model_h.compile(optimizer='adam', loss=jsd_custom_loss, metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [21]:
final = Generate_submission_w2v(test,model_c,model_h)

C:\Users\doudi\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

300

### 7. Generate the submission estimation JSON

In [23]:
import json
import time
import os

path = 'C:/Users/doudi/OneDrive/Documents/ntcir15/eval'
os.chdir(path)
timestr = time.strftime("%Y%m%d%H%M")

In [159]:
with open((timestr + '_' + 'dev_eval.json'), 'w', encoding='utf-8') as f: 
    f.write(json.dumps(final, ensure_ascii=False, indent=2))

In [27]:
# with open(('test_submit_01.json'), 'w', encoding='utf-8') as f: 
#     f.write(json.dumps(final, ensure_ascii=False, indent=2))
import json
p = 'C:\\Users\\doudi\\OneDrive\\Documents\\ntcir15\\Dataset\\New_DialEval-1\\test_cn.json'
t = json.load(open(p, encoding="utf-8"))

In [28]:
len(t)

300

In [160]:
import os
os.chdir('C:\\Users\\doudi\\OneDrive\\Documents\\ntcir15\\eval')
!python eval.py 202007141655_dev_eval.json dev_cn.json

{'quality': None, 'nugget': {'jsd': 4.760648615016886, 'rnss': 3.029105568527165}}


In [1]:
2**(-3.431)

0.09271843336507012

In [2]:
2**(-3.880)

0.06792092890787864